# Test

## Matrix Product State

In [1]:
using CUDA
using ITensors, ITensorMPS, Plots

In [2]:
a = 1
ga = 0.5
ma = 0.25
g = ga/a
m_org = ma/a
m_lat = m_org - a*g^2/8
# x = 1/(a^2*g^2)
# mu = 2*m_lat/(a*g^2)

0.21875

In [3]:
site_test = siteinds("S=1/2", 8)

8-element Vector{Index{Int64}}:
 (dim=2|id=489|"S=1/2,Site,n=1")
 (dim=2|id=709|"S=1/2,Site,n=2")
 (dim=2|id=708|"S=1/2,Site,n=3")
 (dim=2|id=175|"S=1/2,Site,n=4")
 (dim=2|id=36|"S=1/2,Site,n=5")
 (dim=2|id=783|"S=1/2,Site,n=6")
 (dim=2|id=680|"S=1/2,Site,n=7")
 (dim=2|id=930|"S=1/2,Site,n=8")

In [4]:
N = 10
sites = siteinds("S=1/2", N)

os = OpSum()
for n in 1:N-1
    # os += (1/(4*a)), "X", n,"X", n+1
    # os += (1/(4*a)), "Y", n,"Y", n+1
    os += (1/(2*a)), "S+", n,"S-", n+1
    os += (1/(2*a)), "S+", n+1,"S-", n
end

for n in 1:N
    os += (-1)^n*m_lat/2, "Z", n
end

for n in 1:N-1
    for k in 1:n
        os += g^2*a/4*(-1)^k, "Z", k
        os += g^2*a/4, "Id", 1
    end
    for j in 1:n
        for i in 1:j-1
            os += g^2*a/4, "Z", i, "Z", j
            os += g^2*a/4*(-1)^(i+j), "Id", 1
            os += g^2*a/4*(-1)^j, "Z", i
            os += g^2*a/4*(-1)^i, "Z", j
        end
    end
end

H = MPO(os, sites)

MPO
[1] ((dim=5|id=184|"Link,l=1"), (dim=2|id=242|"S=1/2,Site,n=1")', (dim=2|id=242|"S=1/2,Site,n=1"))
[2] ((dim=5|id=184|"Link,l=1"), (dim=5|id=138|"Link,l=2"), (dim=2|id=388|"S=1/2,Site,n=2")', (dim=2|id=388|"S=1/2,Site,n=2"))
[3] ((dim=5|id=138|"Link,l=2"), (dim=5|id=12|"Link,l=3"), (dim=2|id=306|"S=1/2,Site,n=3")', (dim=2|id=306|"S=1/2,Site,n=3"))
[4] ((dim=5|id=12|"Link,l=3"), (dim=5|id=986|"Link,l=4"), (dim=2|id=253|"S=1/2,Site,n=4")', (dim=2|id=253|"S=1/2,Site,n=4"))
[5] ((dim=5|id=986|"Link,l=4"), (dim=5|id=910|"Link,l=5"), (dim=2|id=360|"S=1/2,Site,n=5")', (dim=2|id=360|"S=1/2,Site,n=5"))
[6] ((dim=5|id=910|"Link,l=5"), (dim=5|id=763|"Link,l=6"), (dim=2|id=526|"S=1/2,Site,n=6")', (dim=2|id=526|"S=1/2,Site,n=6"))
[7] ((dim=5|id=763|"Link,l=6"), (dim=5|id=388|"Link,l=7"), (dim=2|id=543|"S=1/2,Site,n=7")', (dim=2|id=543|"S=1/2,Site,n=7"))
[8] ((dim=5|id=388|"Link,l=7"), (dim=5|id=6|"Link,l=8"), (dim=2|id=609|"S=1/2,Site,n=8")', (dim=2|id=609|"S=1/2,Site,n=8"))
[9] ((dim=5|id=6|"L

In [5]:
nsweeps = 30
maxdim = [10,20,100,100,200]
cutoff = [1E-10]

psi0_init = random_mps(sites, linkdims = 10)
# 

MPS
[1] ((dim=2|id=242|"S=1/2,Site,n=1"), (dim=10|id=819|"Link,l=1"))
[2] ((dim=10|id=819|"Link,l=1"), (dim=2|id=388|"S=1/2,Site,n=2"), (dim=10|id=888|"Link,l=2"))
[3] ((dim=10|id=888|"Link,l=2"), (dim=2|id=306|"S=1/2,Site,n=3"), (dim=10|id=340|"Link,l=3"))
[4] ((dim=10|id=340|"Link,l=3"), (dim=2|id=253|"S=1/2,Site,n=4"), (dim=10|id=492|"Link,l=4"))
[5] ((dim=10|id=492|"Link,l=4"), (dim=2|id=360|"S=1/2,Site,n=5"), (dim=10|id=903|"Link,l=5"))
[6] ((dim=10|id=903|"Link,l=5"), (dim=2|id=526|"S=1/2,Site,n=6"), (dim=10|id=375|"Link,l=6"))
[7] ((dim=10|id=375|"Link,l=6"), (dim=2|id=543|"S=1/2,Site,n=7"), (dim=8|id=962|"Link,l=7"))
[8] ((dim=8|id=962|"Link,l=7"), (dim=2|id=609|"S=1/2,Site,n=8"), (dim=4|id=266|"Link,l=8"))
[9] ((dim=4|id=266|"Link,l=8"), (dim=2|id=754|"S=1/2,Site,n=9"), (dim=2|id=986|"Link,l=9"))
[10] ((dim=2|id=986|"Link,l=9"), (dim=2|id=778|"S=1/2,Site,n=10"))


In [9]:
energy0,psi0 = dmrg(H,psi0_init;nsweeps,maxdim,cutoff)

After sweep 1 energy=-3.0981832946028023  maxlinkdim=10 maxerr=9.09E-04 time=0.019
After sweep 2 energy=-3.0987189016978203  maxlinkdim=12 maxerr=9.11E-11 time=0.004
After sweep 3 energy=-3.098718908575706  maxlinkdim=11 maxerr=9.84E-11 time=0.004
After sweep 4 energy=-3.098718908575785  maxlinkdim=11 maxerr=9.82E-11 time=0.011
After sweep 5 energy=-3.0987189085760924  maxlinkdim=11 maxerr=9.67E-11 time=0.004
After sweep 6 energy=-3.0987189085761413  maxlinkdim=11 maxerr=9.67E-11 time=0.004
After sweep 7 energy=-3.0987189085761653  maxlinkdim=11 maxerr=9.67E-11 time=0.004
After sweep 8 energy=-3.0987189085761715  maxlinkdim=11 maxerr=9.67E-11 time=0.007
After sweep 9 energy=-3.0987189085761795  maxlinkdim=11 maxerr=9.67E-11 time=0.004
After sweep 10 energy=-3.098718908576183  maxlinkdim=11 maxerr=9.67E-11 time=0.007
After sweep 11 energy=-3.0987189085761813  maxlinkdim=11 maxerr=9.67E-11 time=0.005
After sweep 12 energy=-3.098718908576186  maxlinkdim=11 maxerr=9.67E-11 time=0.004
After

(-3.098718908576182, MPS
[1] ((dim=2|id=24|"Link,l=1"), (dim=2|id=242|"S=1/2,Site,n=1"))
[2] ((dim=4|id=575|"Link,l=2"), (dim=2|id=388|"S=1/2,Site,n=2"), (dim=2|id=24|"Link,l=1"))
[3] ((dim=2|id=306|"S=1/2,Site,n=3"), (dim=8|id=154|"Link,l=3"), (dim=4|id=575|"Link,l=2"))
[4] ((dim=2|id=253|"S=1/2,Site,n=4"), (dim=11|id=801|"Link,l=4"), (dim=8|id=154|"Link,l=3"))
[5] ((dim=2|id=360|"S=1/2,Site,n=5"), (dim=11|id=455|"Link,l=5"), (dim=11|id=801|"Link,l=4"))
[6] ((dim=2|id=526|"S=1/2,Site,n=6"), (dim=11|id=509|"Link,l=6"), (dim=11|id=455|"Link,l=5"))
[7] ((dim=2|id=543|"S=1/2,Site,n=7"), (dim=8|id=305|"Link,l=7"), (dim=11|id=509|"Link,l=6"))
[8] ((dim=2|id=609|"S=1/2,Site,n=8"), (dim=4|id=435|"Link,l=8"), (dim=8|id=305|"Link,l=7"))
[9] ((dim=2|id=754|"S=1/2,Site,n=9"), (dim=2|id=139|"Link,l=9"), (dim=4|id=435|"Link,l=8"))
[10] ((dim=2|id=778|"S=1/2,Site,n=10"), (dim=2|id=139|"Link,l=9"))
)

In [7]:
# psi_contracted = contract(psi)

In [10]:
psi1_init = random_mps(sites, linkdims = 10)
energy1,psi1 = dmrg(H,[psi0],psi1_init;nsweeps,maxdim,cutoff)

After sweep 1 energy=-2.7427972915481655  maxlinkdim=10 maxerr=5.76E-04 time=0.254
After sweep 2 energy=-2.754175329383427  maxlinkdim=17 maxerr=7.84E-11 time=0.006
After sweep 3 energy=-2.754176009067117  maxlinkdim=13 maxerr=8.45E-11 time=0.020
After sweep 4 energy=-2.7541760099555934  maxlinkdim=13 maxerr=9.04E-11 time=0.006
After sweep 5 energy=-2.754176009954808  maxlinkdim=13 maxerr=7.22E-11 time=0.006
After sweep 6 energy=-2.754176009954813  maxlinkdim=13 maxerr=7.06E-11 time=0.014
After sweep 7 energy=-2.7541760099548105  maxlinkdim=13 maxerr=7.06E-11 time=0.006
After sweep 8 energy=-2.7541760099548127  maxlinkdim=13 maxerr=7.06E-11 time=0.005
After sweep 9 energy=-2.754176009954811  maxlinkdim=13 maxerr=7.06E-11 time=0.009
After sweep 10 energy=-2.7541760099548123  maxlinkdim=13 maxerr=7.06E-11 time=0.005
After sweep 11 energy=-2.7541760099548123  maxlinkdim=13 maxerr=7.06E-11 time=0.008
After sweep 12 energy=-2.7541760099548105  maxlinkdim=13 maxerr=7.06E-11 time=0.005
After 

(-2.7541760099548105, MPS
[1] ((dim=2|id=477|"Link,l=1"), (dim=2|id=242|"S=1/2,Site,n=1"))
[2] ((dim=4|id=963|"Link,l=2"), (dim=2|id=388|"S=1/2,Site,n=2"), (dim=2|id=477|"Link,l=1"))
[3] ((dim=2|id=306|"S=1/2,Site,n=3"), (dim=8|id=272|"Link,l=3"), (dim=4|id=963|"Link,l=2"))
[4] ((dim=2|id=253|"S=1/2,Site,n=4"), (dim=11|id=38|"Link,l=4"), (dim=8|id=272|"Link,l=3"))
[5] ((dim=2|id=360|"S=1/2,Site,n=5"), (dim=13|id=341|"Link,l=5"), (dim=11|id=38|"Link,l=4"))
[6] ((dim=2|id=526|"S=1/2,Site,n=6"), (dim=10|id=332|"Link,l=6"), (dim=13|id=341|"Link,l=5"))
[7] ((dim=2|id=543|"S=1/2,Site,n=7"), (dim=8|id=22|"Link,l=7"), (dim=10|id=332|"Link,l=6"))
[8] ((dim=2|id=609|"S=1/2,Site,n=8"), (dim=4|id=987|"Link,l=8"), (dim=8|id=22|"Link,l=7"))
[9] ((dim=2|id=754|"S=1/2,Site,n=9"), (dim=2|id=174|"Link,l=9"), (dim=4|id=987|"Link,l=8"))
[10] ((dim=2|id=778|"S=1/2,Site,n=10"), (dim=2|id=174|"Link,l=9"))
)

In [11]:
psi1_copy = copy(psi1)
op_sm = op("S-", sites[1])

# try apply op_sm to psi1_copy
apply(op_sm, psi1_copy)

MPS
[1] ((dim=2|id=242|"S=1/2,Site,n=1"), (dim=2|id=477|"Link,l=1"))
[2] ((dim=4|id=963|"Link,l=2"), (dim=2|id=388|"S=1/2,Site,n=2"), (dim=2|id=477|"Link,l=1"))
[3] ((dim=2|id=306|"S=1/2,Site,n=3"), (dim=8|id=272|"Link,l=3"), (dim=4|id=963|"Link,l=2"))
[4] ((dim=2|id=253|"S=1/2,Site,n=4"), (dim=11|id=38|"Link,l=4"), (dim=8|id=272|"Link,l=3"))
[5] ((dim=2|id=360|"S=1/2,Site,n=5"), (dim=13|id=341|"Link,l=5"), (dim=11|id=38|"Link,l=4"))
[6] ((dim=2|id=526|"S=1/2,Site,n=6"), (dim=10|id=332|"Link,l=6"), (dim=13|id=341|"Link,l=5"))
[7] ((dim=2|id=543|"S=1/2,Site,n=7"), (dim=8|id=22|"Link,l=7"), (dim=10|id=332|"Link,l=6"))
[8] ((dim=2|id=609|"S=1/2,Site,n=8"), (dim=4|id=987|"Link,l=8"), (dim=8|id=22|"Link,l=7"))
[9] ((dim=2|id=754|"S=1/2,Site,n=9"), (dim=2|id=174|"Link,l=9"), (dim=4|id=987|"Link,l=8"))
[10] ((dim=2|id=778|"S=1/2,Site,n=10"), (dim=2|id=174|"Link,l=9"))


## Quantum Number Conserving

In [9]:
using BenchmarkTools

In [10]:
@benchmark siteqnc = siteinds("S=1/2", 5, conserve_qns = true)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.356 μs …  4.178 ms  ┊ GC (min … max):  0.00% … 99.32%
 Time  (median):     4.911 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   9.954 μs ± 59.634 μs  ┊ GC (mean ± σ):  11.35% ±  1.99%

  █▁█▇▃▁    ▅▄   ▃▆▅▁▁  ▁                                    ▂
  ██████▇▆▅▇████▇█████▇██▇▇▆▇▇▆▅▅▅▄▆▆▄▅▄▄▄▆▃▄▄▆▅▅▄▅▄▆▆▅▅▃▄▅▅ █
  2.36 μs      Histogram: log(frequency) by time     50.2 μs <

 Memory estimate: 13.68 KiB, allocs estimate: 113.

In [11]:
@benchmark siteqnc = cu(siteinds("S=1/2", 5, conserve_qns = true))

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   9.900 μs …  11.745 ms  ┊ GC (min … max): 0.00% … 69.07%
 Time  (median):     10.767 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.922 μs ± 118.929 μs  ┊ GC (mean ± σ):  5.09% ±  0.69%

  ▆█▄▃▃▂▁  ▁▂▂▂▁▂▂▁         ▁▂▂▃▂                              ▂
  ██████████████████▇▆▃▅▅▅▆███████▇▇█▇▇▆▄▅▅▅▄▃▅▁▅▅▄▅▅▄▄▁▄▃▁▃▃▄ █
  9.9 μs        Histogram: log(frequency) by time      50.5 μs <

 Memory estimate: 13.96 KiB, allocs estimate: 123.

In [12]:
siteqnc = cu(siteinds("S=1/2", 5, conserve_qns = true))
siteqnc

5-element CuArray{Index{Vector{Pair{QN, Int64}}}, 1, CUDA.DeviceMemory}:
 (dim=2|id=10|"S=1/2,Site,n=1") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=804|"S=1/2,Site,n=2") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=686|"S=1/2,Site,n=3") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=271|"S=1/2,Site,n=4") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1
 (dim=2|id=604|"S=1/2,Site,n=5") <Out>
 1: QN("Sz",1) => 1
 2: QN("Sz",-1) => 1

# PDF of Schwinger Model with Time evolving MPS

In [13]:
for j in 1:9-1
    # skip if j is even
    if j%2 != 0
        continue
    end
    println(j)
end

2
4
6
8


In [25]:
function time_evolve_d0(h, e_h, delta_t, site, d, cutoff = 1E-8)
    """
    site: the full site
    delta_t: time step
    d: just the number d in equation 8. In paper, d is even, but here d is odd since there is no site 0.
    h: the hadron state, first eigenstate of the Hamiltonian as MPS.
    e_h: energy eigenvalue of corresponding h state
    delta_t: a number of our choice
    """
    t_d = d/2*delta_t

    # step 0: the e^iHt_d term: H here can be intepreted as e_h
    phase = exp(-im*e_h*t_d)

    # step 1: apply the Pi (k<d) (-i sigma_z^k)
    op_zs = ITensor[]
    for k in 1:d-1
        op_z = -im*op("Z", site[k])
        push!(op_zs, op_z)
    end

    # step 1.5: apply the sigma_d^+ term
    op_sp = op("S+", site[d])

    # step 2: apply the e^-H_d-1 delta_t, e^-H_d-3 delta_t, ... e^-H_3 delta_t, e^-H_1 delta_t term
    gates = ITensor[]
    for j in 1:d-1
        # skip if j is even
        if j%2 == 0
            continue
        end
        s1 = site[j+1] # 1, 3, 5, 7...' th site, but we add 1 to make it 2, 4, 6, 8...
        s2 = site[j+2]

        h_n = 1/(2*a)*op("S+", s1)*op("S-", s2) 
        # + 1/(2*a)*op("S+", s2)*op("S-", s1) 
            + (-1)^j*m_lat/2*op("Z", s1) 
            # + (-1)^(j+1)*m_lat/2*op("Z", s2)
        
        # for k in 1:j
        #     h_n += g^2*a/4*(-1)^k*op("Z", k)
        #     h_n += g^2*a/4*op("Id", 1)
        #     for i in 1:k-1
        #         h_n += g^2*a/4*op("Z", i)*op("Z", k)
        #         h_n += g^2*a/4*(-1)^(i+k)*op("Id", 1)
        #         h_n += g^2*a/4*(-1)^k*op("Z", i)
        #         h_n += g^2*a/4*(-1)^i*op("Z", k)
        #     end
        # end
        G_j = exp(-im*h_n*delta_t)
        push!(gates, G_j)
    end

    # step 3: apply the Pi (k'<0) (i_sigma_z^k') term

    # step 3.5: apply the sigma_0^- term
    op_sm = op("S-", site[1])

    # step final: calculate phase * <h|op_z's, op_sp, gates, op_sm |h>
    # phase is step0, op_z's is step1, op_sp is step1.5, gates is step2, op_sm is step3.5.
    h_transformed = phase * h
    # h_transformed = apply(op_sm, h_transformed; cutoff)
    # for g in gates
    #     h_transformed = apply(h_transformed, g)
    # end
    # h_transformed = apply(op_sp, h_transformed; cutoff)
    # for op_z in reverse(op_zs)
    #     h_transformed = apply(h_transformed, op_z)
    # end
    
    # expectation value of h
    return inner(h, h_transformed)

end
    

time_evolve_d0 (generic function with 2 methods)

In [27]:
h_input = psi1
e_h_input = energy1
delta_t_input = 0.1
site_input = sites
d_input = 5

time_evolve_d0(h_input, e_h_input, delta_t_input, site_input, d_input)

0.7721719937296565 + 0.6354135756336724im